# Tutorial 0: Curate Raw RNA-seq Data

In this tutorial we will download publicly available raw RNA-sequencing data, in the form of fastqs, from the Cancer Cell Line Encyclopedia (CCLE) Sequence Read Archive (SRA) repository. This CCLE expression dataset is for use in the demo, and will serve as the basis for generating raw data Quilt packages, and input to Nextflow nf-core pipelines. If available, you can sub in your in-house datasets in place of the demo CCLE dataset for a more practical use case tailored to your company's platform. Although this demo will be centered around an RNA-seq expression dataset and the corresponding `nf-core/rna-seq` pipeline, the principles and methods underlying the demo are generalizable across -omics datasets. 

# 1. Set Up


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import quilt3 as quilt3
import numpy as np
import glob
import os
import tqdm as tqdm

# 2. Load RNA s

First, we downloaded 